In [1]:
from wtforms import Form

from wtforms.ext.sqlalchemy.orm import model_form
from dash_deep.app import scripts_db_models
from dash_deep.widjets.widjets_factory import generate_widjet_from_form
from dash_deep.utils import generate_wtform_instances_and_input_form_widjets
from dash_deep.app import db

from dash_deep.scripts.semantic_segmentation_train_script import run
import click

from dash_deep.cli.cli_factory import generate_click_cli

# Decide on the model saving/init and so on logic
# start with bash?

def function_to_be_wrapped(**kwargs):
    
    print(kwargs)

from dash_deep.app import scripts_db_models

from dash_deep.scripts.semantic_segmentation_train_script import PascalSegmentation

forms, widjets = generate_wtform_instances_and_input_form_widjets([PascalSegmentation])

form = forms[0]

cli = generate_click_cli(form, function_to_be_wrapped)

/home/daniil/anaconda2/lib/python2.7/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning:

SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.



In [1]:
from dash_deep.index import wtform_classes

In [14]:
from dash_deep.index import script_files_title_names, script_files_url_endpoints

In [15]:
script_files_title_names

['Pascal Semantic Segmentation Experiment',
 'Imagenet Classification Experiment']

In [16]:
script_files_url_endpoints

['/scripts/pascal_semantic_segmentation_experiment',
 '/scripts/imagenet_classification_experiment']

In [3]:
wtform_class = wtform_classes[0]

In [13]:
wtform_class.actions

{'main': <function dash_deep.scripts.semantic_segmentation_train_script.run>}